#### **BigBasket Product Query Engine  Notebook 1**
**It contains**
1. smooth data transformation 
2. extracted important meta data,  encoding done using sentence encoder and stored in dict object along with meta data  
3. returns space.pkl which will use to store context in vector db(qdrant) file containg vector representation of context and important meta data

In [1]:
!pip install -q qdrant-client
!pip install -q sentence-transformers
!pip install -q torch

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.1 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.1 which is incompatible.
google-cloud-aiplatform 0.6.0a1 requires google-api-core[grpc]<2.0.0dev,>=1.22.2, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.11.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires protobuf<4.0.0dev,>=3.12.0, but you have protobuf 4.25.1 which is incompatible.
google-cloud-bigtable 1.7.3 requires protobuf<4.0.0dev, but you have protobuf 4.25.1 which is incompatible.
google-cloud-datastore 

In [2]:
import torch
import pandas as pd
import sys
import json
import warnings
warnings.filterwarnings('ignore')
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
from tqdm.auto import tqdm
from typing import List



In [3]:
CSV_PATH =  "/kaggle/input/chaabi/bigBasketProducts.csv"
df = pd.read_csv(CSV_PATH).drop(columns = ["index"])

In [4]:
df

,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.00,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.00,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.00,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.00,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.00,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...
...,...,...,...,...,...,...,...,...,...
27550,"Wottagirl! Perfume Spray - Heaven, Classic",Beauty & Hygiene,Fragrances & Deos,Layerr,199.20,249.0,Perfume,3.9,Layerr brings you Wottagirl Classic fragrant b...
27551,Rosemary,Gourmet & World Food,Cooking & Baking Needs,Puramate,67.50,75.0,"Herbs, Seasonings & Rubs",4.0,Puramate rosemary is enough to transform a dis...
27552,Peri-Peri Sweet Potato Chips,Gourmet & World Food,"Snacks, Dry Fruits, Nuts",FabBox,200.00,200.0,Nachos & Chips,3.8,We have taken the richness of Sweet Potatoes (...
27553,Green Tea - Pure Original,Beverages,Tea,Tetley,396.00,495.0,Tea Bags,4.2,"Tetley Green Tea with its refreshing pure, ori..."


In [5]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
    
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
# retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [6]:
class make_embedding_ds(torch.utils.data.IterableDataset):
    def __init__(self, csv = df):
        """
        will compute embedding using sentence encoder
        
        """
        super(make_embedding_ds).__init__()
        self.csv = csv
        self.total_row = csv.shape[0]
        self.col = csv.columns.to_list()

    def __iter__(self):
        for row_no in range(self.total_row):
            
            #testing: make comment it later
       
            
            
            row = self.csv.iloc[row_no].to_dict()
            product_name = row[self.col[0]]
            story = f"{row[ self.col[0] ]} is of category {row[self.col[1]]} and sub category is {row[self.col[2]]}. {row[self.col[0]]} is type {row[self.col[6]]}. brand of {row[self.col[0]]} is {row[self.col[3]]}, with rating {row[self.col[7]]}. sale price of {row[self.col[0]]} is {row[self.col[4]]} with market price {row[self.col[5]]}, description of {row[self.col[0]]} is {row[self.col[8]]}"
            emb = retriever.encode(story)
            
            
            yield row_no,product_name,story,emb
    

ds = make_embedding_ds(df)


dataloader = torch.utils.data.DataLoader(ds, num_workers=1,batch_size=128)


In [7]:
json_emb = {
    "payload":[],
    "emb": []
}
batch_no = 1
for row,product_name,story,emb in dataloader:
    
    _batch_len = len(row)
    
    
    
    print(f"batch no-{batch_no} competed")
    batch_no += 1
    
    for index in range(_batch_len):
        
        temp = {}
        temp["id"] = row[index].item()
        temp["product"] = product_name[index]
        temp["story"] = story[index]
        
        json_emb["payload"].append(temp)
        json_emb["emb"].append(emb[index].numpy().tolist())
        
    

    
#     json_emb["payload"].append(batch[0])
#     json_emb["emb"].append(batch[1][0])

    


batch no-1 competed
batch no-2 competed
batch no-3 competed
batch no-4 competed
batch no-5 competed
batch no-6 competed
batch no-7 competed
batch no-8 competed
batch no-9 competed
batch no-10 competed
batch no-11 competed
batch no-12 competed
batch no-13 competed
batch no-14 competed
batch no-15 competed
batch no-16 competed
batch no-17 competed
batch no-18 competed
batch no-19 competed
batch no-20 competed
batch no-21 competed
batch no-22 competed
batch no-23 competed
batch no-24 competed
batch no-25 competed
batch no-26 competed
batch no-27 competed
batch no-28 competed
batch no-29 competed
batch no-30 competed
batch no-31 competed
batch no-32 competed
batch no-33 competed
batch no-34 competed
batch no-35 competed
batch no-36 competed
batch no-37 competed
batch no-38 competed
batch no-39 competed
batch no-40 competed
batch no-41 competed
batch no-42 competed
batch no-43 competed
batch no-44 competed
batch no-45 competed
batch no-46 competed
batch no-47 competed
batch no-48 competed
b

In [8]:
sys.getsizeof(json_emb)/1024

0.2265625

In [9]:
import pickle

# Deserialize the data from the pickle file
with open('space.pkl', 'wb') as file:
    pickle.dump(json_emb, file)